In [10]:
!pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.38
    Uninstalling SQLAlchemy-2.0.38:
      Successfully uninstalled SQLAlchemy-2.0.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

# Load the dataset
dataset = load_dataset("OpenAssistant/oasst1")

# Filter English messages
english_data = dataset['train'].filter(lambda x: x['lang'] == 'en')
print(f"✅ English Dataset Loaded: {len(english_data)} samples")

# Extract prompt-response pairs
pairs = []
for conversation in english_data:
    if isinstance(conversation, dict) and 'messages' in conversation:
        messages = conversation['messages']
        for i in range(len(messages) - 1):
            if messages[i]['role'] == 'user' and messages[i + 1]['role'] == 'assistant':
                pairs.append({
                    'prompt': messages[i]['text'],
                    'response': messages[i + 1]['text'],
                    'quality': messages[i + 1].get('quality', 0)  # Default quality if missing
                })

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Example: GPT-2 model
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

# Tokenize dataset
tokenized_pairs = []
for pair in pairs:
    tokenized_input = tokenizer(pair['prompt'], truncation=True, padding='max_length', max_length=128)
    tokenized_output = tokenizer(pair['response'], truncation=True, padding='max_length', max_length=128)
    tokenized_pairs.append({
        'input_ids': tokenized_input['input_ids'],
        'attention_mask': tokenized_input['attention_mask'],
        'labels': tokenized_output['input_ids'],
        'quality': pair['quality']
    })

# Create a PyTorch dataset
class PreferenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['labels']),
            'quality': torch.tensor(item['quality'])
        }

# Initialize dataset
train_dataset = PreferenceDataset(tokenized_pairs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
import torch
import argparse
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
#from dataset import PreferenceDataset
from loss import dpo_loss
from clean_dataset import clean_raw_data
from copy import deepcopy
import matplotlib.pyplot as plt
from datasets import load_from_disk
from transformers import DataCollatorForLanguageModeling
from peft import LoraConfig

def load_model_and_tokenizer(model_name: str, device: str) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    """
    Loads the supervised fine-tuned language model (sft) and tokenizer from huggingface.

    Args:
        model_name: The name or path of the sft model.
        device: The device to load the model onto.

    Returns:
        model: The loaded language model.
        tokenizer: The corresponding tokenizer.
    """
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Ensure the tokenizer has a padding token (important for batching)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load the language model
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Move the model to the specified device
    model.to(device)

    return model, tokenizer

class DPOTrainer:
    def __init__(self, model, ref_model, peft_config, args, train_dataset, eval_dataset, tokenizer):
        self.model = model
        self.ref_model = ref_model
        self.peft_config = peft_config
        self.args = args
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.tokenizer = tokenizer

        self.data_collator = DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=args.learning_rate)

        # Exclude reference model parameters from optimization
        for param in self.ref_model.parameters():
            param.requires_grad = False

        self.train_dataloader = DataLoader(self.train_dataset, batch_size=args.batch_size, shuffle=True, collate_fn=self.data_collator)
        self.eval_dataloader = DataLoader(self.eval_dataset, batch_size=args.batch_size, shuffle=False, collate_fn=self.data_collator)

    def train(self):
        self.model.train()
        train_losses = []
        eval_losses = []

        for epoch in range(self.args.num_train_epochs):
            total_loss = 0.0

            print(f"\nEpoch {epoch + 1}/{self.args.num_train_epochs}")

            for step, batch in enumerate(self.train_dataloader):
                self.optimizer.zero_grad()

                # Compute the DPO loss
                loss = dpo_loss(self.model, self.ref_model, batch, self.args.beta, self.args.device)

                # Backpropagation
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

                # Logging
                if (step + 1) % 10 == 0 or (step + 1) == len(self.train_dataloader):
                    avg_loss = total_loss / (step + 1)
                    print(f"  Step {step + 1}/{len(self.train_dataloader)}, Loss: {avg_loss:.4f}")

            # Evaluate the model on the eval set
            self.model.eval()
            with torch.no_grad():
                eval_loss = 0.0
                for eval_batch in self.eval_dataloader:
                    eval_loss += dpo_loss(self.model, self.ref_model, eval_batch, self.args.beta, self.args.device).item()
                avg_eval_loss = eval_loss / len(self.eval_dataloader)

            # Epoch summary
            avg_loss = total_loss / len(self.train_dataloader)
            train_losses.append(avg_loss)
            eval_losses.append(avg_eval_loss)
            print(f"Epoch {epoch + 1} completed. Average training Loss: {avg_loss:.4f}. Average eval Loss: {avg_eval_loss:.4f}")

        # Save the fine-tuned model
        self.model.save_pretrained(self.args.output_dir)
        self.tokenizer.save_pretrained(self.args.output_dir)
        print(f"\nModel saved to {self.args.output_dir}")

        # Plot the training and eval loss
        epochs = range(1, self.args.num_train_epochs + 1)
        plt.figure(figsize=(8, 6))
        plt.plot(epochs, train_losses, 'b-', label='Training Loss')
        plt.plot(epochs, eval_losses, 'r-', label='Eval Loss')
        plt.title('Training and Eval Loss over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.savefig('loss_plot.png')
        plt.show()

def main():
    """
    Main function to execute the training loop for DPO.
    """

    # Argument parsing
    parser = argparse.ArgumentParser(description='Train a language model using Direct Preference Optimization.')
    parser.add_argument('--model_name', type=str, default='gpt2', help='The name of the pre-trained language model.')
    parser.add_argument('--learning_rate', type=float, default=5e-5, help='The learning rate for the optimizer.')
    parser.add_argument('--num_train_epochs', type=int, default=6, help='The number of training epochs.')
    parser.add_argument('--beta', type=float, default=0.1, help='The hyperparameter beta for DPO.')
    parser.add_argument('--batch_size', type=int, default=5, help='The batch size for training.')
    parser.add_argument('--max_length', type=int, default=512, help='The maximum sequence length for tokenization.')
    parser.add_argument('--output_dir', type=str, default='output_model', help='The directory to save the fine-tuned model.')
    parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu', help='The device to run the training on.')
    args = parser.parse_args()

    # Device configuration
    device = torch.device(args.device)
    print(f"Using device: {device}")

    # Load the tokenized datasets
    train_tokenized = load_from_disk('train_tokenized')
    val_tokenized = load_from_disk('val_tokenized')

    # Load the model and tokenizer
    model, tokenizer = load_model_and_tokenizer(args.model_name, device)

    # Create a reference model
    ref_model = deepcopy(model)
    ref_model.to(device)
    ref_model.eval()

    # LoRA config based on QLoRA paper & Sebastian Raschka experiment
    peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
    )

    # Training Arguments
    training_args = argparse.Namespace(
        output_dir=args.output_dir,
        eval_strategy="epoch",
        learning_rate=args.learning_rate,
        per_device_train_batch_size=args.batch_size,
        per_device_eval_batch_size=args.batch_size,
        num_train_epochs=args.num_train_epochs,
        weight_decay=0.01,
        fp16=True,  # Use mixed precision training
        gradient_accumulation_steps=4,  # Accumulate gradients
        dataloader_num_workers=4,  # Use multiple workers for data loading
        logging_dir='./logs',  # Directory for storing logs
        logging_steps=10,  # Log every 10 steps
        beta=args.beta,
        device=args.device
    )

    # Initialize the DPOTrainer
    trainer = DPOTrainer(
        model=model,
        ref_model=ref_model,
        peft_config=peft_config,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
    )

    # Start training
    trainer.train()

if __name__ == '__main__':
    main()

ImportError: cannot import name 'dpo_loss' from 'loss' (/usr/local/lib/python3.11/dist-packages/loss/__init__.py)

In [9]:


!pip install datasets transformers torch peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

# ✅ Load the dataset
dataset = load_dataset("OpenAssistant/oasst1")

# ✅ Filter English messages
english_data = dataset['train'].filter(lambda x: x['lang'] == 'en')
print(f"✅ English Dataset Loaded: {len(english_data)} samples")

✅ English Dataset Loaded: 39283 samples


In [13]:
# ✅ Extract prompt-response pairs
pairs = []
for conversation in english_data:
    if isinstance(conversation, dict) and 'messages' in conversation:
        messages = conversation['messages']
        for i in range(len(messages) - 1):
            if messages[i]['role'] == 'user' and messages[i + 1]['role'] == 'assistant':
                pairs.append({
                    'prompt': messages[i]['text'],
                    'response': messages[i + 1]['text'],
                    'quality': messages[i + 1].get('quality', 0)  # Default quality if missing
                })
# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Example: GPT-2 model
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

# ✅ Tokenize dataset
tokenized_pairs = []
for pair in pairs:
    tokenized_input = tokenizer(pair['prompt'], truncation=True, padding='max_length', max_length=128)
    tokenized_output = tokenizer(pair['response'], truncation=True, padding='max_length', max_length=128)
    tokenized_pairs.append({
        'input_ids': tokenized_input['input_ids'],
        'attention_mask': tokenized_input['attention_mask'],
        'labels': tokenized_output['input_ids'],
        'quality': pair['quality']
    })
# ✅ Create a PyTorch dataset
class PreferenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['labels']),
            'quality': torch.tensor(item['quality'])
        }

# ✅ Initialize dataset
train_dataset = PreferenceDataset(tokenized_pairs)

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# ✅ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()  # Free CUDA memory
print(f"🔹 Using device: {device}")

🔹 Using device: cuda


In [15]:
# ✅ Load dataset
dataset = load_dataset("openai/summarize_from_feedback", "axis", split="test")  # Use test split
print(f"✅ Dataset Loaded: {len(dataset)} samples")

README.md:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

summarize_from_feedback.py:   0%|          | 0.00/9.38k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.53M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.07M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/6312 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8585 [00:00<?, ? examples/s]

✅ Dataset Loaded: 6312 samples


In [16]:
# ✅ Load a pre-trained Transformer model (GPT-2 for reward modeling)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Fix the tokenizer padding issue (GPT models don’t have a default pad token)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding token
reward_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
reward_model.config.pad_token_id = tokenizer.pad_token_id  # Ensure model uses pad_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# ✅ Preprocessing function
def preprocess_data(example):
    """Processes dataset to extract chosen and rejected responses for reward training."""
    return {
        "prompt": str(example.get("info", {}).get("post", "")),  # Convert to string
        "chosen": str(example.get("summary", "")),  # Preferred summary
        "rejected": str(example.get("summary", ""))  # Duplicate since dataset only has 'summary'
    }

# ✅ Apply preprocessing safely
dataset = dataset.map(preprocess_data, remove_columns=["info", "summary", "worker", "batch", "split"], features=None)
# ✅ Convert dataset to list format
prompts = [entry["prompt"] for entry in dataset]
chosen_responses = [entry["chosen"] for entry in dataset]
rejected_responses = [entry["rejected"] for entry in dataset]

# ✅ Tokenize inputs
chosen_outputs = tokenizer(chosen_responses, padding=True, truncation=True, max_length=128, return_tensors="pt")
rejected_outputs = tokenizer(rejected_responses, padding=True, truncation=True, max_length=128, return_tensors="pt")

# ✅ Ensure tensors are non-empty
print("Chosen Outputs Shape:", chosen_outputs["input_ids"].shape)
print("Rejected Outputs Shape:", rejected_outputs["input_ids"].shape)

Map:   0%|          | 0/6312 [00:00<?, ? examples/s]

Chosen Outputs Shape: torch.Size([6312, 128])
Rejected Outputs Shape: torch.Size([6312, 128])


In [18]:
# ✅ Define optimizer & loss function
optimizer = optim.AdamW(reward_model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

# ✅ Ensure labels match the shape of chosen/rejected outputs
num_samples = len(chosen_outputs["input_ids"])
labels = torch.cat([
    torch.ones(num_samples, dtype=torch.float32),
    torch.zeros(num_samples, dtype=torch.float32)
]).to(device)
BATCH_SIZE = 4

# ✅ Training function
def train_reward_model(model, optimizer, chosen_outputs, rejected_outputs, labels, epochs=3):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        for i in range(0, len(chosen_outputs["input_ids"]), BATCH_SIZE):
            # Select batch
            batch_chosen = {k: v[i:i+BATCH_SIZE].to(device) for k, v in chosen_outputs.items()}
            batch_rejected = {k: v[i:i+BATCH_SIZE].to(device) for k, v in rejected_outputs.items()}
            batch_labels = labels[i:i+BATCH_SIZE]

            # ✅ Ensure label shape matches model output
            chosen_scores = model(**batch_chosen).logits.squeeze()
            rejected_scores = model(**batch_rejected).logits.squeeze()

            # ✅ Check for empty tensors before computing loss
            if chosen_scores.shape[0] == 0 or rejected_scores.shape[0] == 0:
                print("Skipping empty batch...")
                continue

            # ✅ Fix label slicing to match logits shape
            loss = criterion(chosen_scores, batch_labels[:chosen_scores.shape[0]]) + criterion(rejected_scores, batch_labels[:rejected_scores.shape[0]])

            loss.backward()
            optimizer.step()

            # Free unused memory
            torch.cuda.empty_cache()

        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# ✅ Train the reward model
train_reward_model(reward_model, optimizer, chosen_outputs, rejected_outputs, labels)

Epoch 1, Loss: 0.0000
Epoch 2, Loss: 0.0000
Epoch 3, Loss: 0.0000


In [19]:
# ✅ Save trained reward model
torch.save(reward_model.state_dict(), "reward_model.pth")
print("✅ Reward model training complete & saved!")

✅ Reward model training complete & saved!


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset

# ✅ Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device: {device}")

🔹 Using device: cuda


In [21]:
# ✅ Load trained reward model from Task 2
model_name = "gpt2"
reward_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
reward_model.load_state_dict(torch.load("reward_model.pth", map_location=device))
reward_model.config.pad_token_id = reward_model.config.eos_token_id  # Ensure pad token is set

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-21-ebf3a24a5f3c>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only

In [22]:
# ✅ Load dataset (Using OpenAI Summarization Feedback dataset)
dataset = load_dataset("openai/summarize_from_feedback", "axis", split="test")  # Use test split
print(f"✅ Dataset Loaded: {len(dataset)} samples")

# ✅ Load tokenizer & set padding
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding token

✅ Dataset Loaded: 6312 samples


In [23]:
# ✅ Preprocessing function for DPO
def preprocess_data(example):
    """Processes dataset for DPO training."""
    return {
        "prompt": str(example.get("info", {}).get("post", "")),  # Extract original post
        "chosen": str(example.get("summary", "")),  # Preferred summary
        "rejected": str(example.get("summary", ""))  # Currently duplicating since dataset only has 'summary'
    }

# ✅ Apply preprocessing
dataset = dataset.map(preprocess_data, remove_columns=["info", "summary", "worker", "batch", "split"], features=None)
# ✅ Convert dataset to list format
prompts = [entry["prompt"] for entry in dataset]
chosen_responses = [entry["chosen"] for entry in dataset]
rejected_responses = [entry["rejected"] for entry in dataset]

# ✅ Tokenize inputs
chosen_outputs = tokenizer(chosen_responses, padding=True, truncation=True, max_length=128, return_tensors="pt")
rejected_outputs = tokenizer(rejected_responses, padding=True, truncation=True, max_length=128, return_tensors="pt")

# ✅ Convert dataset to PyTorch tensors
inputs = tokenizer(prompts, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
chosen_inputs = {k: v.to(device) for k, v in chosen_outputs.items()}
rejected_inputs = {k: v.to(device) for k, v in rejected_outputs.items()}
# ✅ Define DPO loss function
def dpo_loss(chosen_scores, rejected_scores, beta=0.1):
    """
    Implements Direct Preference Optimization (DPO) loss.

    Args:
    - chosen_scores: Model outputs for chosen responses
    - rejected_scores: Model outputs for rejected responses
    - beta: Temperature parameter for softmax scaling

    Returns:
    - Loss value
    """
    logits_diff = chosen_scores - rejected_scores
    loss = -torch.log(torch.sigmoid(logits_diff / beta)).mean()
    return loss
# ✅ Define optimizer
optimizer = optim.AdamW(reward_model.parameters(), lr=1e-5)

# ✅ Reduce batch size to prevent CUDA OOM
BATCH_SIZE = 4

# ✅ Training function using DPO
def train_dpo(model, optimizer, chosen_inputs, rejected_inputs, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        for i in range(0, len(chosen_inputs["input_ids"]), BATCH_SIZE):
            # Select batch
            batch_chosen = {k: v[i:i+BATCH_SIZE] for k, v in chosen_inputs.items()}
            batch_rejected = {k: v[i:i+BATCH_SIZE] for k, v in rejected_inputs.items()}

            # Compute logits
            chosen_scores = model(**batch_chosen).logits.squeeze()
            rejected_scores = model(**batch_rejected).logits.squeeze()

            # Ensure correct tensor dimensions
            if chosen_scores.shape[0] == 0 or rejected_scores.shape[0] == 0:
                print("Skipping empty batch...")
                continue

            # Compute DPO loss
            loss = dpo_loss(chosen_scores, rejected_scores)

            loss.backward()
            optimizer.step()

            # Free unused memory
            torch.cuda.empty_cache()

        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# ✅ Train the model with DPO
train_dpo(reward_model, optimizer, chosen_inputs, rejected_inputs)

Map:   0%|          | 0/6312 [00:00<?, ? examples/s]

Epoch 1, Loss: 1.2536
Epoch 2, Loss: 0.6967
Epoch 3, Loss: 0.4340
Epoch 4, Loss: 0.4708
Epoch 5, Loss: 0.7259
Epoch 6, Loss: 0.8413
Epoch 7, Loss: 0.7990
Epoch 8, Loss: 0.8456
Epoch 9, Loss: 0.6662
Epoch 10, Loss: 0.9444


In [24]:
# ✅ Save fine-tuned reward model
torch.save(reward_model.state_dict(), "reward_model_dpo.pth")
print("✅ DPO Fine-Tuning Complete & Model Saved!")

✅ DPO Fine-Tuning Complete & Model Saved!
